<a href="https://colab.research.google.com/github/SARA3SAEED/aws-ML/blob/main/10_3_Appendix_Empirical_Performance_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating AI Models: Code, Human, and Model-Based Grading

In this notebook, we'll delve into a trio of widely-used techniques for assessing the effectiveness of AI models, like Claude v3:

1. Code-based grading
2. Human grading
3. Model-based grading

We'll illustrate each approach through examples and examine their respective advantages and limitations, when gauging AI performance.

## Code-Based Grading Example: Sentiment Analysis

In this example, we'll evaluate Claude's ability to classify the sentiment of movie reviews as positive or negative. We can use code to check if the model's output matches the expected sentiment.

In [ ]:
# Import python's built-in regular expression library
import re

# Import boto3 and json
import boto3
import json
from botocore.exceptions import ClientError
# Store the model name and AWS region for later use

%store -r modelId
%store -r region

In [ ]:
# Function to build the input prompt for sentiment analysis
def build_input_prompt(review):
    user_content = f"""Classify the sentiment of the following movie review as either 'positive' or 'negative' provide only one of those two choices:
    <review>{review}</review>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "review": "This movie was amazing! The acting was superb and the plot kept me engaged from start to finish.",
        "golden_answer": "positive"
    },
    {
        "review": "I was thoroughly disappointed by this film. The pacing was slow and the characters were one-dimensional.",
        "golden_answer": "negative"
    }
]

# Function to get completions from the model
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

def get_completion(messages, system_prompt=None):
    inference_config = {
        "temperature": 0.5,
        "maxTokens": 200
    }
    additional_model_fields = {
        "top_p": 1
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
        "inferenceConfig": inference_config,
        "additionalModelRequestFields": additional_model_fields
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["review"])) for item in eval]

# Print the outputs and golden answers
for output, question in zip(outputs, eval):
    print(f"Review: {question['review']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n")

# Function to grade the completions
def grade_completion(output, golden_answer):
    return output.lower() == golden_answer.lower()

# Grade the completions and print the accuracy
grades = [grade_completion(output, item["golden_answer"]) for output, item in zip(outputs, eval)]
print(f"Accuracy: {sum(grades) / len(grades) * 100}%")

Review: This movie was amazing! The acting was superb and the plot kept me engaged from start to finish.
Golden Answer: positive
Output: positive

Review: I was thoroughly disappointed by this film. The pacing was slow and the characters were one-dimensional.
Golden Answer: negative
Output: negative

Accuracy: 100.0%


## Human Grading Example: Essay Scoring

Some tasks, like scoring essays, are difficult to evaluate with code alone. In this case, we can provide guidelines for human graders to assess the model's output.

In [ ]:
# Function to build the input prompt for essay generation
def build_input_prompt(topic):
    user_content = f"""Write a short essay discussing the following topic:
    <topic>{topic}</topic>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "topic": "The importance of education in personal development and societal progress",
        "golden_answer": "A high-scoring essay should have a clear thesis, well-structured paragraphs, and persuasive examples discussing how education contributes to individual growth and broader societal advancement."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["topic"])) for item in eval]

# Print the outputs and golden answers
for output, item in zip(outputs, eval):
    print(f"Topic: {item['topic']}\n\nGrading Rubric:\n {item['golden_answer']}\n\nModel Output:\n{output}\n")

Topic: The importance of education in personal development and societal progress

Grading Rubric:
 A high-scoring essay should have a clear thesis, well-structured paragraphs, and persuasive examples discussing how education contributes to individual growth and broader societal advancement.

Model Output:
Here is a short essay discussing the importance of education in personal development and societal progress:

The Importance of Education in Personal Development and Societal Progress

Education is one of the most powerful tools we have for personal growth and societal advancement. At the individual level, education helps us develop the knowledge, skills, and critical thinking abilities to reach our full potential. At the societal level, an educated populace is essential for driving innovation, economic prosperity, and positive social change.

On a personal level, education empowers us to better understand ourselves and the world around us. Through learning, we gain new perspectives, d

## Model-Based Grading Examples

We can use Claude to grade its own outputs by providing the model's response and a grading rubric. This allows us to automate the evaluation of tasks that would typically require human judgment.

### Example 1: Summarization

In this example, we'll use Claude to assess the quality of a summary it generated. This can be useful when you need to evaluate the model's ability to capture key information from a longer text concisely and accurately. By providing a rubric that outlines the essential points that should be covered, we can automate the grading process and quickly assess the model's performance on summarization tasks.

In [ ]:
# Function to build the input prompt for summarization
def build_input_prompt(text):
    user_content = f"""Please summarize the main points of the following text:
    <text>{text}</text>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Function to build the grader prompt for assessing summary quality
def build_grader_prompt(output, rubric):
    user_content = f"""Assess the quality of the following summary based on this rubric:
    <rubric>{rubric}</rubric>
    <summary>{output}</summary>
    Provide a score from 1-5, where 1 is poor and 5 is excellent."""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "text": "The Magna Carta, signed in 1215, was a pivotal document in English history. It limited the powers of the monarchy and established the principle that everyone, including the king, was subject to the law. This laid the foundation for constitutional governance and the rule of law in England and influenced legal systems worldwide.",
        "golden_answer": "A high-quality summary should concisely capture the key points: 1) The Magna Carta's significance in English history, 2) Its role in limiting monarchical power, 3) Establishing the principle of rule of law, and 4) Its influence on legal systems around the world."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["text"])) for item in eval]

# Grade the completions
grades = [get_completion(build_grader_prompt(output, item["golden_answer"])) for output, item in zip(outputs, eval)]

# Print the summary quality score
print(f"Summary quality score: {grades[0]}")

Summary quality score: Based on the provided rubric, I would rate the quality of the summary as a 4 out of 5.

The summary effectively captures the key points:

1. The Magna Carta's significance in English history - This is covered in the first point about the Magna Carta being a pivotal document in English history.

2. Its role in limiting monarchical power - This is clearly stated in the second point about the Magna Carta limiting the powers of the monarchy.

3. Establishing the principle of rule of law - This is also covered in the second point, which mentions that the Magna Carta established the principle that everyone, including the king, was subject to the law.

4. Its influence on legal systems around the world - This is addressed in the fourth point about the Magna Carta influencing legal systems worldwide.

The summary is concise and effectively captures the key points. The only minor suggestion would be to potentially include


### Example 2: Fact-Checking

In this example, we'll use Claude to fact-check a claim and then assess the accuracy of its fact-checking. This can be useful when you need to evaluate the model's ability to distinguish between accurate and inaccurate information. By providing a rubric that outlines the key points that should be covered in a correct fact-check, we can automate the grading process and quickly assess the model's performance on fact-checking tasks.

In [ ]:
# Function to build the input prompt for fact-checking
def build_input_prompt(claim):
    user_content = f"""Determine if the following claim is true or false:
    <claim>{claim}</claim>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Function to build the grader prompt for assessing fact-check accuracy
def build_grader_prompt(output, rubric):
    user_content = f"""Based on the following rubric, assess whether the fact-check is correct:
    <rubric>{rubric}</rubric>
    <fact-check>{output}</fact-check>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "claim": "The Great Wall of China is visible from space.",
        "golden_answer": "A correct fact-check should state that this claim is false. While the Great Wall is an impressive structure, it is not visible from space with the naked eye."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["claim"])) for item in eval]

grades = []
for output, item in zip(outputs, eval):
    # Print the claim, fact-check, and rubric
    print(f"Claim: {item['claim']}\n")
    print(f"Fact-check: {output}]\n")
    print(f"Rubric: {item['golden_answer']}\n")

    # Grade the fact-check
    grader_prompt = build_grader_prompt(output, item["golden_answer"])
    grade = get_completion(grader_prompt)
    grades.append("correct" in grade.lower())

# Print the fact-checking accuracy
accuracy = sum(grades) / len(grades)
print(f"Fact-checking accuracy: {accuracy * 100}%")

Claim: The Great Wall of China is visible from space.

Fact-check: The claim "The Great Wall of China is visible from space" is false.

This claim is a common misconception that has been debunked by many space agencies and scientific authorities. The Great Wall of China, despite its impressive size, is not visible to the naked eye from space.

Here are the key reasons why this claim is false:

1. Scale and width: The Great Wall of China is relatively narrow, with an average width of only 4-5 meters (13-16 feet). From the distance of space, it is simply too thin to be easily distinguishable.

2. Contrast and camouflage: The Great Wall blends in with the surrounding terrain, making it difficult to distinguish from the ground. The wall's color and materials used in its construction make it visually blend in with the natural environment.

3. Atmospheric conditions: The atmosphere and curvature of the Earth make it challenging to see small man-]

Rubric: A correct fact-check should state th

### Example 3: Tone Analysis

In this example, we'll use Claude to analyze the tone of a given text and then assess the accuracy of its analysis. This can be useful when you need to evaluate the model's ability to identify and interpret the emotional content and attitudes expressed in a piece of text. By providing a rubric that outlines the key aspects of tone that should be identified, we can automate the grading process and quickly assess the model's performance on tone analysis tasks.

In [ ]:
# Function to build the input prompt for tone analysis
def build_input_prompt(text):
    user_content = f"""Analyze the tone of the following text:
    <text>{text}</text>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Function to build the grader prompt for assessing tone analysis accuracy
def build_grader_prompt(output, rubric):
    user_content = f"""Assess the accuracy of the following tone analysis based on this rubric:
    <rubric>{rubric}</rubric>
    <tone-analysis>{output}</tone-analysis>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "text": "I can't believe they canceled the event at the last minute. This is completely unacceptable and unprofessional!",
        "golden_answer": "The tone analysis should identify the text as expressing frustration, anger, and disappointment. Key words like 'can't believe', 'last minute', 'unacceptable', and 'unprofessional' indicate strong negative emotions."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["text"])) for item in eval]

# Grade the completions
grades = [get_completion(build_grader_prompt(output, item["golden_answer"])) for output, item in zip(outputs, eval)]

# Print the tone analysis quality
print(f"Tone analysis quality: {grades[0]}")

Tone analysis quality: Based on the provided rubric, the tone analysis appears to be accurate.

The key points highlighted in the rubric are:

1. The tone analysis should identify the text as expressing frustration, anger, and disappointment.
2. The use of words like "can't believe", "last minute", "unacceptable", and "unprofessional" indicate strong negative emotions.

The tone analysis correctly identifies the tone as one of frustration and annoyance. It accurately notes the use of the phrase "I can't believe," which suggests disbelief and surprise. Additionally, the analysis correctly interprets the language used, such as "completely unacceptable and unprofessional," as conveying a strong negative reaction and a sense of the speaker being upset or angry.

The analysis also correctly summarizes the overall tone as one of criticism and disapproval, reflecting the speaker's strong feelings of dissatisfaction and unhappiness with the


These examples demonstrate how code-based, human, and model-based grading can be used to evaluate AI models like Claude on various tasks. The choice of evaluation method depends on the nature of the task and the resources available. Model-based grading offers a promising approach for automating the assessment of complex tasks that would otherwise require human judgment.